# Nonparametric Bayesian label prediction on a graph

This script generates the plots and parts of the storyline for the presentation of the doctoral thesis "Nonparametric Bayesian label prediction on a graph" that is comprehensible to non-specialists. It includes an analysis of simulated station traffic data on the Transport for London network (tube, overground, DLR, tramlink). The edges in the network are actual line connections and stations that are within walking distance as per the [TfL network])(http://content.tfl.gov.uk/large-print-tube-map.pdf) (May 2019).

* The data folder contains the edge list of the network and station coordinates and zones in .csv format. The Lines.xlsx file contains the edge list in .xlsx format with a separate sheet per line.
* This notebook contains the storyline, plots and example analysis.

In [1]:
import csv  # read station coordinates and zones
from matplotlib.colors import LinearSegmentedColormap  # to make custom RAG colormap
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from scipy.sparse import diags  # sparse diagonal matrices
from scipy.sparse.linalg import eigsh  # eigendecomposition

## Introduction

My research is about label prediction on a graph. A graph, or network, I encounter or hear about almost every day is the tube in London. This presentation will introduce some of the concepts related to my thesis to give you some idea of its contents.

Let's have a look at the TfL (Transport for London) network. This contains the tube, DLR, cable car, overground, TfL rail and trams.